In [1]:
#364130 to ...
BLOCKS_INIT = '0000000013ab9f8ed78b254a429d3d5ad52905362e01bf6c682940337721eb51' #pizza

In [2]:
import os.path
import urllib.request
import json
import pickle
from selenium import webdriver
from time import sleep

In [3]:
!mkdir data

mkdir: data: File exists


In [4]:
def find_block_info(HASH):
    if not os.path.isfile('data/'+BLOCK_HASH+'.json'):
        block_info = urllib.request.urlopen("https://blockchain.info/rawblock/" + BLOCK_HASH)
        block_info_json = json.load(block_info)
        with open('data/'+BLOCK_HASH+'.json', 'w') as outfile:
            json.dump(block_info_json, outfile)
        return block_info_json
    else:
        return json.load(open('data/'+BLOCK_HASH+'.json','r'))

In [5]:
BLOCK_HASH = BLOCKS_INIT
for a in range(2):
    print(BLOCK_HASH)
    block_info_json = find_block_info(BLOCK_HASH)
    BLOCK_HASH = block_info_json['prev_block']

0000000013ab9f8ed78b254a429d3d5ad52905362e01bf6c682940337721eb51
00000000152340ca42227603908689183edc47355204e7aca59383b0aaac1fd8


In [6]:
import random

In [7]:
def render_graph_and_save_screenshot(graph,BLOCK_HASH):
    with open('20nov.json', 'w') as outfile:
        json.dump(graph, outfile)
    browser = webdriver.Chrome()
    browser.implicitly_wait(1)
    browser.get('about:blank')
    browser.maximize_window()
    sleep(1)
    browser.save_screenshot('blank.png')
    browser.set_window_position(0, 0)
    browser.set_window_size(1000, 1000)
    browser.get('http://localhost:8002/code/20nov.html')
    sleep(80)
    browser.save_screenshot(BLOCK_HASH+'-'+str(random.random())+'.png')
    browser.quit()

In [8]:
def create_graph(block_info_json):
    nodes_input = []
    nodes_transaction = []
    nodes_output = []
    links = []
    for transaction in block_info_json['tx']:
        nodes_transaction.append(transaction['hash'])
        for input in transaction['inputs']:
            if 'prev_out' in input:
                prev_out = input['prev_out']
                if 'addr' in prev_out:
                    address = prev_out['addr']
                    
                    if address not in remap_outputs_dict:#totally new input
                        address += '-' + str(random.random())
                        nodes_input.append(address)  
                        links.append({
                            'source':address,
                            'target':transaction['hash'],
                            'value':1
                        })
                    elif address in remap_outputs_dict:
                        new_address = remap_outputs_dict[address]
                        del remap_outputs_dict[address]
#                         nodes_input.append(new_address)  
                        links.append({
                            'source':new_address,
                            'target':transaction['hash'],
                            'value':1
                        })
                    
        for output in transaction['out']:
            if 'addr' in output:
                address_out = output['addr']
                address_out_unique = address_out + '-' + str(random.random())
                remap_outputs_dict[address_out] = address_out_unique
                nodes_output.append(address_out_unique)  
                links.append({
                    'source':transaction['hash'],
                    'target':address_out_unique,
                    'value':1
                })

                    
                    
                
    nodes_json = [{'id':a,'group':1} for a in nodes_input] + \
        [{'id':a,'group':2} for a in nodes_transaction] + \
        [{'id':a,'group':3} for a in nodes_output]
    links_json = links
    output = {
        'nodes':nodes_json,
        'links':links
    }
    
    return output

In [9]:
BLOCK_HASH = BLOCKS_INIT
for a in range(0,100):
    remap_outputs_dict = {}
    block_info_json = find_block_info(BLOCK_HASH)
    output = create_graph(block_info_json)
#     if(a>=10):
    render_graph_and_save_screenshot(output,BLOCK_HASH)
    BLOCK_HASH = block_info_json['prev_block']


KeyboardInterrupt: 